### - Sobhan Moradian Daghigh
#### - 5/29/2022
#### - ANN - HW06

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import urllib
import tensorflow as tf, keras
from sklearn.metrics import accuracy_score
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers.merge import concatenate

import warnings
warnings.filterwarnings("ignore")

import itertools

%load_ext tensorboard

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tb_callback = keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/logs', histogram_freq=1)

In [ ]:
class EarlyStoppingCallback(keras.callbacks.Callback):
  def __init__(self, patience=0):
    super(EarlyStoppingCallback, self).__init__()
    self.patience = patience

  def on_train_begin(self, logs=None):
    self.best = 0
    self.wait = 0
  
  def on_epoch_end(self, epoch, logs=None):
    current_acc = logs.get('accuracy')
    if np.less(self.best, current_acc):
      self.best = current_acc
      self.wait = 0
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if np.greater_equal(self.wait, self.patience):
        self.model.stop_training = True
        self.model.set_weights(self.best_weights)

In [ ]:
es_callback = EarlyStoppingCallback(patience=3)

# Load Data

In [53]:
import urllib

In [96]:
def load_data(base_url):
  datas = []

  with urllib.request.urlopen(base_url + 'names.txt') as names:
    idx = 0
    for name in names.readlines():
      name = name.decode("utf-8").split('\n')[0]
      data = pd.read_csv(base_url + name, delimiter=',', encoding='utf-16-le')
      if not data.empty:
        datas.append(data)
        print('   |_ {}- {} loaded.'.format(idx, name))
        idx += 1

  return datas

In [97]:
base_url = 'https://raw.githubusercontent.com/nahbos/AUT-Neural-Networks/main/HW06/dataset/'
datas = load_data(base_url)

   |_ 0- IRX6XAFF0005.csv loaded.
   |_ 1- IRX6XALS0002.csv loaded.
   |_ 2- IRX6XS300003.csv loaded.
   |_ 3- IRX6XSLC0000.csv loaded.
   |_ 4- IRX6XSNT0009.csv loaded.
   |_ 5- IRX6XTAL0001.csv loaded.
   |_ 6- IRX6XTPI0009.csv loaded.
   |_ 7- IRX6XTPI0009[2022-05-23-10-18-57].csv loaded.
   |_ 8- IRX6XTPI0025.csv loaded.
   |_ 9- IRX6XWAI0001.csv loaded.
   |_ 10- IRXYXTPI0009.csv loaded.
   |_ 11- IRXYXTPI0025.csv loaded.
   |_ 12- IRXZXCMI0000.csv loaded.
   |_ 13- IRXZXENG0009.csv loaded.
   |_ 14- IRXZXOBM0003.csv loaded.
   |_ 15- IRXZXOCI0008.csv loaded.
   |_ 16- IRXZXTRI0006.csv loaded.
   |_ 17- IRXZXWDI0007.csv loaded.


In [108]:
datas[np.random.randint(len(datas))].head()

,<TICKER>,<DTYYYYMMDD>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,<OPENINT>,<OPENINT>.1,<OPENINT>.2,<COl12>,<COl13>,<LAST>
0,شاخص50شركت_فعالتر6,20081206,370.2,370.2,367.8,367.8,1689796,3.022612e+09,379,0.0,Top 50 Index,شاخص50شركت_فعالتر,367.8
1,شاخص50شركت_فعالتر6,20081207,367.8,367.8,364.2,364.2,3309008,6.070659e+09,586,367.8,Top 50 Index,شاخص50شركت_فعالتر,364.2
2,شاخص50شركت_فعالتر6,20081208,362.2,362.2,361.5,361.5,4889069,7.873401e+09,945,364.2,Top 50 Index,شاخص50شركت_فعالتر,361.5
3,شاخص50شركت_فعالتر6,20081210,359.5,359.5,356.1,356.1,8583982,1.627066e+10,709,361.5,Top 50 Index,شاخص50شركت_فعالتر,356.1
4,شاخص50شركت_فعالتر6,20081213,354.2,354.2,351.1,351.1,26202686,5.026677e+10,2010,356.1,Top 50 Index,شاخص50شركت_فعالتر,351.1


## LSTM